In [107]:
import pandas as pd
import numpy as np
import plotly.express as px
import pandas as pd

In [108]:
df = pd.read_csv('../data/10_monthly_returns_complete.csv')
df_close = pd.read_csv('../data/10_monthly_adjacent_close.csv', index_col=0)
df_overview = pd.read_csv('../data/data_10_overview.csv', index_col=0)

## Equities: Risk-Return

In [109]:
# Create a scatter plot
fig = px.scatter(df_overview[['stock_ticker_symbol', 'return_rate_5y_avg', 'volatility_5y']], 
                 x='volatility_5y', 
                 y='return_rate_5y_avg', 
                 text='stock_ticker_symbol',
                 title='Risk-Return Plot',
                 labels={'Risk (Standard Deviation)': 'Risk (Standard Deviation)',
                         'Expected Return': 'Expected Return'},
                 template='plotly')

# Add labels to the points
fig.update_traces(textposition='top center')

# Show the plot
fig.show()

## Equities: Sharpe-Ratio

In [110]:
df_overview.head(2)

,company_name,industry,stock_exchange,stock_ticker_symbol,market_capital_euro,score,return_rate_1y_avg,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg,volatility_1y,volatility_5y,volatility_10y,volatility_25y
0,"Kimball Electronics, Inc.",Technology Hardware,NAS,KE,5.481967e+08,0.000120,23.898182,20.615169,18.074612,18.074612,0.075240,0.121466,0.108617,0.108617
1,"Steelcase, Inc.",Commercial Services,NYS,SCS,1.461633e+09,0.013235,12.520981,10.730252,11.347063,8.444393,0.090186,0.118895,0.108954,0.106236


In [111]:
# Define the risk-free rate
risk_free_rate = 0.02

# Calculate the excess return
df_overview['excess_return'] = df_overview['return_rate_5y_avg'] - risk_free_rate

# Calculate the Sharpe Ratio
df_overview['sharpe_ratio'] = df_overview['excess_return'] / df_overview['volatility_5y']


In [118]:
df_overview

,company_name,industry,stock_exchange,stock_ticker_symbol,market_capital_euro,score,return_rate_1y_avg,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg,volatility_1y,volatility_5y,volatility_10y,volatility_25y,excess_return,sharpe_ratio
0,"Kimball Electronics, Inc.",Technology Hardware,NAS,KE,5.481967e+08,0.000120,23.898182,20.615169,18.074612,18.074612,0.075240,0.121466,0.108617,0.108617,20.595169,169.554659
1,"Steelcase, Inc.",Commercial Services,NYS,SCS,1.461633e+09,0.013235,12.520981,10.730252,11.347063,8.444393,0.090186,0.118895,0.108954,0.106236,10.710252,90.081846
2,HNI Corp.,Commercial Services,NYS,HNI,2.138245e+09,0.015005,NaN,NaN,NaN,NaN,0.089088,0.087072,0.096107,0.101771,NaN,NaN
3,"Avnet, Inc.",Technology Hardware,NAS,AVT,4.517528e+09,0.018784,NaN,NaN,NaN,NaN,0.064486,0.093569,0.081193,0.108015,NaN,NaN
4,ACCO Brands Corp.,Commercial Services,NYS,ACCO,4.399426e+08,0.019449,5.249025,6.045924,6.931758,7.810648,0.093098,0.135272,0.123144,0.249993,6.025924,44.546852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,"Halozyme Therapeutics, Inc.",Pharmaceuticals,NAS,HALO,6.410791e+09,0.249868,NaN,NaN,NaN,NaN,0.116140,0.129228,0.143864,0.193229,NaN,NaN
955,Matthews International Corp.,Commercial Services,NAS,MATW,7.831761e+08,0.250174,NaN,NaN,NaN,NaN,0.098510,0.098342,0.089769,0.075885,NaN,NaN
956,"KVH Industries, Inc. (Delaware)",Technology Hardware,NAS,KVHI,8.416451e+07,0.250012,4.900909,9.089661,9.936471,10.322667,0.073585,0.096023,0.098989,0.143995,9.069661,94.452709
957,"NeoGenomics, Inc.",Pharmaceuticals,NAS,NEO,1.821173e+09,0.250415,14.838182,25.395085,17.712689,9.420861,0.125032,0.167200,0.144416,0.170306,25.375085,151.765073


In [131]:
# Plot the Sharpe Ratio distribution using Plotly
fig = px.scatter(df_overview,
                   x='volatility_5y',
                   y='return_rate_5y_avg',
                   title='Sharpe Ratio Distribution',
                   labels={'volatility_5y': 'Volatility', 'return_rate_5y_avg': 'Return Rate'},
                   template='plotly_white')

# Show the plot
fig.show()

In [113]:
# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Melt the returns dataframe to long format
df_long_returns = df.melt(id_vars=['Date'], var_name='stock_ticker_symbol', value_name='return_rate_5y')

# Merge the returns and industries dataframes
df_merged = pd.merge(df_long_returns, df_overview, on='stock_ticker_symbol')

# Resample to monthly and calculate mean return for each ticker
df_merged.set_index('Date', inplace=True)

df_i_share_ratio = df_merged[['industry', 'return_rate_5y', 'volatility_5y']].groupby(['industry']).resample('ME').mean()
df_i_share_ratio

return_rate_5y  volatility_5y
industry        Date                                     
Auto Components 1999-08-31             NaN       0.159585
                1999-09-30        0.905000       0.159585
                1999-10-31        0.905000       0.159585
                1999-11-30        1.045000       0.159585
                1999-12-31        0.964167       0.159585
...                                    ...            ...
Utilities       2024-03-31        1.035000       0.100341
                2024-04-30        0.975000       0.100341
                2024-05-31        1.125000       0.100341
                2024-06-30        0.962000       0.100341
                2024-07-31        1.101000       0.100341

[10500 rows x 2 columns]

Creating separate columns from indices

In [114]:
df_i_share_ratio = df_i_share_ratio.reset_index(level=0)
df_i_share_ratio = df_i_share_ratio.reset_index(level=0)

In [115]:
df_i_share_ratio[['industry', 'Date', 'return_rate_5y']]

,industry,Date,return_rate_5y
0,Auto Components,1999-08-31,NaN
1,Auto Components,1999-09-30,0.905000
2,Auto Components,1999-10-31,0.905000
3,Auto Components,1999-11-30,1.045000
4,Auto Components,1999-12-31,0.964167
...,...,...,...
10495,Utilities,2024-03-31,1.035000
10496,Utilities,2024-04-30,0.975000
10497,Utilities,2024-05-31,1.125000
10498,Utilities,2024-06-30,0.962000


In [117]:
# Create line plot
fig = px.line(df_i_share_ratio, 
              x='Date', 
              y='return_rate_5y',
              color='industry', 
              title='Industry historical returns',
              labels={'Date': 'Date', 'return_rate_5y': 'Return Rate'})

# Show the plot
fig.show()